In [7]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [12]:
# Parâmetros
BATCH_SIZE = 32
IMAGE_SIZE = 224  # Redefinindo tamanho da imagem para o padrão de redes convulacionais (CNNs)
DATA_DIR = '/Users/rafaelmacedo/Documents/Code/eixo5_grupo5_20251/projeto/data'  # Path to your dataset folder
TRAIN_FOLDER = "Training"
TEST_FOLDER = "Testing"

# Transformações e enriquecimento de dados
train_transforms = transforms.Compose([
    # Converte imagens preto e branco para os canais RGB (vermelho, verde e azul). 
    # Isto é necessário pois as redes convulacionais esperam esses canais como input.
    transforms.Grayscale(num_output_channels=3),

    # Fixa as dimensões da imagem.
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    
    # Espelha a imagem para esquerda ou direita aleatoriamente
    transforms.RandomHorizontalFlip(),

    # Rotaciona a imagem em +- 10 graus aleatoriamente
    transforms.RandomRotation(10),

    # Converte para a estrutura de dados do PyTorch (tensor)
    transforms.ToTensor(),

    # Esta transformada consiste em escalonar os píxeis para média centrada no 0 e variância unitária.
    # Utiliza-se a média para cada canal de cor (vermelho, verde e azul).
    # Os valores de média e desvio-padrão são usados com base no projeto ImageNet.
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Datasets
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, TRAIN_FOLDER), transform=train_transforms)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, TEST_FOLDER), transform=test_transforms)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Nome das classes
class_names = train_dataset.classes
print(f"Classes: {class_names}")


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [13]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5712
    Root location: /Users/rafaelmacedo/Documents/Code/eixo5_grupo5_20251/projeto/data/Training
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )